In [1]:
import numpy as np
import pandas as pd
import scipy.optimize
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [36]:
def agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"
    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def resid(param, c, p_active):
    return p_active - agonist_only_p_active_theory(c, *param)



In [63]:
#set up plot
from bokeh.plotting import curdoc 
curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[L-Quis]M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

ligand_smooth=np.logspace(-12,-4,200)
p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -9.6, -7, -4))
p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, -9.3, -7.6, -4))

bokeh.io.show(p);

In [38]:
df = pd.read_excel('./MOR_Gi1_normalizeto100.xlsx')
df

,conc,morphine,fentanyl,PZM21,FH210
0,1.000000e-05,82.007061,98.310324,47.978995,51.586148
1,1.999862e-06,89.541212,97.626209,55.077800,60.524068
2,4.000369e-07,89.487904,83.952791,37.148652,50.564417
3,8.000184e-08,67.765027,84.130483,30.378578,39.973959
4,1.599926e-08,40.142767,63.153913,32.244346,31.320347
5,3.199632e-09,36.055845,25.394310,24.301503,6.354586
6,6.400295e-10,11.676469,2.791856,-3.401607,-4.975960
7,1.279971e-10,-2.893407,-5.070137,-7.027417,-7.109156
8,2.558586e-11,-8.702167,8.078201,-11.229839,-14.005392
9,5.116818e-12,-7.623575,2.623049,5.483894,-26.358557


In [75]:
curdoc().clear()
p = bokeh.plotting.figure(plot_height=600,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')
colors=bokeh.palettes.d3['Category10'][10]

for i,ligand in enumerate(df.columns[1:]):
    c, p_active = df['conc'].values, df[ligand].values/100
    p.circle(c, p_active, size=7, color=colors[i],legend_label=ligand)

bokeh.io.show(p);

In [78]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-7,-4,-5])
c, p_active = df['conc'].values, df['morphine'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(res.x)

[-9.6270621  -7.09411868 -3.96001784]


In [54]:
res.x

array([-9.6270621 , -7.09411868, -3.96001784])

In [79]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-4,-4])
c, p_active = df['conc'].values, df['fentanyl'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(res.x)

[-9.82170484 -6.74944575 -4.1378976 ]


In [67]:
res.x

array([-9.82170484, -6.74944575, -4.1378976 ])

In [82]:
def partial_agonist_only_p_active_theory(c_agonist, log_Kd_active, log_Kd_inactive, beta_deltaE):
    "theoretical curve for mglur5 active probability"

    Kd_active = 10**(log_Kd_active)
    Kd_inactive = 10**(log_Kd_inactive)
    a = (1 + c_agonist/Kd_active)

    b = (1 + c_agonist/Kd_inactive)

    return a/(a+b*np.exp(-beta_deltaE))

def partial_resid(param, c, p_active):
    return p_active - partial_agonist_only_p_active_theory(c, *param)

In [83]:
curdoc().clear()
#initial parameter for optimization
p0 = np.array([-9,-7,-4])
c, p_active = df['conc'].values, df['FH210'].values/100

#optimize kd and energy difference to fit the data
res = scipy.optimize.least_squares(partial_resid, p0, args = (c,p_active))

curdoc().clear()
p = bokeh.plotting.figure(plot_height=300,
                         plot_width=600,
                         x_axis_label='[Ligands] M',
                          y_axis_label='active_probability',
                          x_axis_type='log')

p.line(ligand_smooth, partial_agonist_only_p_active_theory(ligand_smooth, res.x[0],res.x[1],res.x[2]))
p.circle(c, p_active, size=7, color=colors[i])

bokeh.io.show(p);
print(res.x)

[-25.79181677  -7.50970466 -41.84699177]


In [73]:
res.x

array([-9.31274888, -7.46314733, -4.        ])